In [1]:
import pyspark 
from pyspark import SparkContext 
sc = SparkContext()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('yarn').appName('').getOrCreate()       

In [3]:
airports = spark.read.csv("/example/airport-codes-na.txt", header = True, inferSchema = True, sep='\t')
airports.createOrReplaceTempView("airports")

In [4]:
airports.show()

+-----------+-----+-------+----+
|       City|State|Country|IATA|
+-----------+-----+-------+----+
| Abbotsford|   BC| Canada| YXX|
|   Aberdeen|   SD|    USA| ABR|
|    Abilene|   TX|    USA| ABI|
|      Akron|   OH|    USA| CAK|
|    Alamosa|   CO|    USA| ALS|
|     Albany|   GA|    USA| ABY|
|     Albany|   NY|    USA| ALB|
|Albuquerque|   NM|    USA| ABQ|
| Alexandria|   LA|    USA| AEX|
|  Allentown|   PA|    USA| ABE|
|   Alliance|   NE|    USA| AIA|
|     Alpena|   MI|    USA| APN|
|    Altoona|   PA|    USA| AOO|
|   Amarillo|   TX|    USA| AMA|
|Anahim Lake|   BC| Canada| YAA|
|  Anchorage|   AK|    USA| ANC|
|   Appleton|   WI|    USA| ATW|
|     Arviat|  NWT| Canada| YEK|
|  Asheville|   NC|    USA| AVL|
|      Aspen|   CO|    USA| ASE|
+-----------+-----+-------+----+
only showing top 20 rows



In [5]:
flightPerf = spark.read.csv("/example/departuredelays.csv", header = True)
flightPerf.createOrReplaceTempView("FlightPerformance")


In [7]:
# 워싱턴 주에 있는 공항은?
spark.sql("select distinct city from airports where state='WA'").show()

+-----------+
|       city|
+-----------+
|      Pasco|
|  Wenatchee|
| Bellingham|
|     Yakima|
|    Seattle|
|    Pullman|
|    Spokane|
| Moses Lake|
|Walla Walla|
+-----------+



In [8]:
# 워싱턴 주에 있는 공항에서 출발한 건을 나열하라
spark.sql("select distinct a.state, a.City, f.origin, f.destination from FlightPerformance f join airports a on a.IATA = f.origin where a.State = 'WA'").collect()


[Row(state='WA', City='Seattle', origin='SEA', destination='PHL'),
 Row(state='WA', City='Seattle', origin='SEA', destination='DEN'),
 Row(state='WA', City='Seattle', origin='SEA', destination='SMF'),
 Row(state='WA', City='Seattle', origin='SEA', destination='HDN'),
 Row(state='WA', City='Seattle', origin='SEA', destination='LAS'),
 Row(state='WA', City='Seattle', origin='SEA', destination='RNO'),
 Row(state='WA', City='Seattle', origin='SEA', destination='JAC'),
 Row(state='WA', City='Seattle', origin='SEA', destination='BOS'),
 Row(state='WA', City='Seattle', origin='SEA', destination='JFK'),
 Row(state='WA', City='Seattle', origin='SEA', destination='OAK'),
 Row(state='WA', City='Seattle', origin='SEA', destination='ABQ'),
 Row(state='WA', City='Seattle', origin='SEA', destination='LAX'),
 Row(state='WA', City='Spokane', origin='GEG', destination='SEA'),
 Row(state='WA', City='Seattle', origin='SEA', destination='SBA'),
 Row(state='WA', City='Seattle', origin='SEA', destination='KO

In [10]:
# 위 건의 연착시간의 합을 출발지 별로 구하시오
spark.sql("select a.City, f.origin, sum(f.delay) as Delays from FlightPerformance f join airports a on a.IATA = f.origin where a.State = 'WA' group by a.City, f.origin order by sum(f.delay) desc").show()


+-------+------+--------+
|   City|origin|  Delays|
+-------+------+--------+
|Seattle|   SEA|159086.0|
|Spokane|   GEG| 12404.0|
|  Pasco|   PSC|   949.0|
+-------+------+--------+

